# Resource planning

In [ ]:
%%html
<style>
table {float:left}
</style>

| Document info | |
| --- | --- | 
| Area of interest: | Cape Town |
| Planning type: | All REL type producers |
| Prepared by: | Waste Labs (wastelabs.co) |
| Prepared for: | Johan W. Joubert |
| Contact: | elias@wastelabs.co |

In [1]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

# Show all code cells outputs
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

import logging

logging.basicConfig(level=logging.INFO)

import pickle

import chart_studio.plotly as py
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot

init_notebook_mode(connected=True)

import cufflinks as cf

cf.go_offline(connected=True)
cf.set_config_file(colorscale="plotly", world_readable=True)

import os
import sys

import geopandas as gpd
import ipywidgets as widgets
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from GPSOdyssey import Kepler
from IPython.display import clear_output
from ipywidgets import fixed, interact, interact_manual, interactive
from keplergl import KeplerGl
from shapely import wkt

# sys.path.insert(0, '../../../mcarptif/')
# sys.path.insert(0, os.path.abspath('../../collection_diagnostics/'))
# sys.path.insert(0, os.path.abspath('../../OSM_processing/'))
import utils.process_gdf as process_gdf

# Extra options
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 1000

from mcarptif.osmnx_network_extract.extract_grptif import NetworkExtract
from mcarptif.osmnx_network_extract.network_code import create_gdf, required_arc_plot
from mcarptif.solver.solve import solve_store_instance
from mcarptif.visualise.route_tables import RouteSummary
from utils.gdf_helpers import create_gdf


def df_style(df):
    if df[0] == "Total":
        return ["font-weight: bold"] * len(df)
    else:
        return [""] * len(df)

/Users/ejwillemse/opt/anaconda3/envs/python_RD/lib/python3.8/site-packages/tables/__init__.py:99: DeprecationWarning:

`np.typeDict` is a deprecated alias for `np.sctypeDict`.



In [2]:
networkh5_file = "../data/05_model_input/sp_files/road_network_simplified_24645.h"
test_network = catalog.load("road_network_simplified_24645_edges")
test_network = test_network.drop(columns=["Unnamed: 0"], errors="ignore")
test_network_directed = catalog.load("road_network_simplified_24645_edges_directed")
nodes = catalog.load("road_network_simplified_24645_nodes")

2022-04-12 11:59:48,136 - kedro.io.data_catalog - INFO - Loading data from `road_network_simplified_24645_edges` (GeoJSONDataSet)...
2022-04-12 11:59:49,730 - kedro.io.data_catalog - INFO - Loading data from `road_network_simplified_24645_edges_directed` (GeoJSONDataSet)...
2022-04-12 11:59:50,653 - kedro.io.data_catalog - INFO - Loading data from `road_network_simplified_24645_nodes` (GeoJSONDataSet)...


In [3]:
test_network.shape

(22189, 19)

In [4]:
%%time
test_network = process_gdf.process_edges(test_network)
test_network.drop_duplicates(["geom_id"], inplace=True)
nodes.sp_index = nodes.index
test_network["arc_index"] = test_network.index
nodes["sp_index"] = nodes.index

CPU times: user 772 ms, sys: 14.8 ms, total: 786 ms
Wall time: 793 ms


/Users/ejwillemse/opt/anaconda3/envs/python_RD/lib/python3.8/site-packages/geopandas/geodataframe.py:199: UserWarning:

Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access



## Service area

In [5]:
%reload_kedro
df_producer_geo = catalog.load("population_sample_network_match")
df_producer_geo = df_producer_geo.merge(
    test_network[["arc_id", "geom_id", "geom_id_inv", "geom_id_order", "arc_index"]],
    validate="m:1",
)
df_producer_geo["land_use"] = df_producer_geo["mainDwellingType"].fillna("nan")

2022-04-12 11:59:52,346 - kedro.framework.session.store - INFO - `read()` not implemented for `BaseSessionStore`. Assuming empty store.
2022-04-12 11:59:52,422 - root - INFO - ** Kedro project Demand estimation and waste collection routing optimisation for the City of Cape Town
2022-04-12 11:59:52,423 - root - INFO - Defined global variable `context`, `session` and `catalog`
2022-04-12 11:59:52,432 - root - INFO - Registered line magic `run_viz`
2022-04-12 11:59:52,433 - kedro.io.data_catalog - INFO - Loading data from `population_sample_network_match` (CSVDataSet)...


## Add service time and demand

In [6]:
df_producer_geo_waste = catalog.load("waste_demand_estimate")
df_producer_geo_waste = df_producer_geo_waste.loc[df_producer_geo_waste["id"] != 0.01]
df_producer_geo_waste.head()
routing_parameters = catalog.load(
    "routing_parameters"
)  # pd.read_excel('../routing_parameters/refuse_parameters.xlsx', sheet_name = 'Refuse_P1')
routing_parameters.fillna(0, inplace=True)

2022-04-12 11:59:52,665 - kedro.io.data_catalog - INFO - Loading data from `waste_demand_estimate` (CSVDataSet)...


,id,waste
0,94369.0,5.874585
1,95457.0,5.884442
2,99366.0,8.942040
3,129493.0,6.901659
4,129494.0,8.191077


2022-04-12 11:59:52,923 - kedro.io.data_catalog - INFO - Loading data from `routing_parameters` (ExcelDataSet)...


In [7]:
calculate = ["bin_service_cost"]
df_producer_geo.drop(columns=calculate, inplace=True, errors="ignore")
df_producer_geo["Domestic"] = 1
df_producer_geo["Non Domestic"] = 0
df_producer_geo["H/M"] = 0
df_producer_geo["category"] = "Landed"
df_producer_geo["land_use"] = "Landed"
df_producer_geo["total units"] = 1
df_producer_geo["total_people"] = 1
df_producer_geo["Postal Code"] = df_producer_geo["id"]

df_producer_geo = df_producer_geo.merge(routing_parameters)
df_producer_geo = df_producer_geo.merge(
    df_producer_geo_waste, how="left", validate="1:1"
)
df_producer_geo["demand"] = df_producer_geo["waste"]
print("\n# offloads (12t)")
df_producer_geo.groupby("land_use")["demand"].sum() / 12000  # number of offloads
print("\n# units")
df_producer_geo.groupby("land_use")["total units"].sum()
print("\n# shifts (bin service cost)")
df_producer_geo.groupby("land_use")["bin_service_cost"].sum() / (
    3600 * 8
)  # number of shifts
df_producer_geo.shape


# offloads (12t)


land_use
Landed    7.602915
Name: demand, dtype: float64


# units


land_use
Landed    13024
Name: total units, dtype: int64


# shifts (bin service cost)


land_use
Landed    2.713333
Name: bin_service_cost, dtype: float64

(13024, 60)

In [8]:
m1 = Kepler(
    data={"Producers": df_producer_geo.drop(columns="geometry")},
    height=800,
    config_path="config/rel_producers.config",
)

2022-04-12 11:59:53,725 - root - INFO - Start preparation of render parameters...
2022-04-12 11:59:53,727 - root - INFO - Loading map config from file: config/rel_producers.config
2022-04-12 11:59:53,749 - root - INFO - Next columns are converted to string: ['pipedWater', 'geom_id_order', 'geom_id', 'geometry_arc_snap_point', 'housingType', 'notes', 'mainDwellingType', 'carAccess', 'toilet', 'geometry_arc', 'category', 'dwellingTenure', 'arc_id', 'geom_id_inv', 'highway', 'land_use', 'geometry_u']
2022-04-12 11:59:53,750 - root - INFO - Start renderding KeplerGL map...
User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


In [9]:
m1.get_render()

KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 'girbu4a', 'type': …

In [10]:
m1.save_config(path="config/rel_producers.config", overwrite_config=True)

2022-04-12 11:59:55,452 - root - WARNING - File already exists and will be over-written.


## Select infrastructure

In [11]:
infrastructure = catalog.load("infrastructure_sample_network_match")
key_pois = infrastructure
key_pois["Postal Code"] = key_pois["id"]

key_pois["u"] = key_pois["arc_u"]
key_pois["v"] = key_pois["arc_v"]
key_pois["key"] = 0
key_pois["geometry_point"] = key_pois["geometry"]
key_pois["geometry"] = key_pois["geometry_arc"]
key_pois = process_gdf.process_edges(key_pois)
key_pois["geometry"] = key_pois["geometry_point"]
key_pois = key_pois.merge(test_network[["arc_id", "arc_index"]], validate="1:1")

2022-04-12 11:59:55,514 - kedro.io.data_catalog - INFO - Loading data from `infrastructure_sample_network_match` (CSVDataSet)...


Possible depot locations:

In [12]:
offload = key_pois.loc[(key_pois["type"] != "depot")]
offload[["description", "type", "arc_index"]]

,description,type,arc_index
2,Offload 1,waste_treatment,16002
3,Offload 2,waste_treatment,18170
4,Offload 3,waste_treatment,17421


Select depot:

In [102]:
key_depos = key_pois.loc[key_pois["type"] == "depot"]
key_depos[["description", "type", "arc_index"]]
key_pois_list = list(key_depos["description"].unique())
key_pois_list


description = "Depot 2"

key_depot_select = key_depos.copy()
key_depot_select = key_depot_select.loc[key_depot_select["description"] == description]
depot_arc_index = key_depot_select.iloc[0]["arc_index"]
display(key_depot_select[["description", "type"]])

,description,type,arc_index
0,Depot 1,depot,3259
1,Depot 2,depot,10907


['Depot 1', 'Depot 2']

,description,type
1,Depot 2,depot


Select treatment infrastructure:

In [113]:
offload = key_pois.loc[key_pois["type"] != "depot"]

descriptions = ["Offload 2", "Offload 3"]

key_if_select = offload.copy()
key_if_select = key_if_select.loc[key_if_select["description"].isin(descriptions)]
if_arc_index = key_if_select["arc_index"].values
display(key_if_select[["description", "type"]])

,description,type
3,Offload 2,waste_treatment
4,Offload 3,waste_treatment


## Select producers

In [114]:
key_list = list(df_producer_geo["category"].unique())

descriptions = ["Landed"]

bad_arcs = ['[(18.4571747, -33.9584372), (18.457166, -33.9585368), (18.457156, -33.9585885), (18.4571405, -33.9586324), (18.4571204, -33.9586875), (18.4570842, -33.9587648), (18.4568647, -33.9591331), (18.4568633, -33.9591415), (18.4568634, -33.9591465), (18.4568651, -33.9591518), (18.4568693, -33.9591571), (18.4568826, -33.9591636), (18.4570165, -33.9592207), (18.4570237, -33.9592212), (18.4570302, -33.9592194), (18.4570376, -33.9592149), (18.4570447, -33.9592066), (18.4570542, -33.9591921), (18.457325, -33.958752), (18.4573407, -33.9587173), (18.4573555, -33.9586744), (18.4573665, -33.9586219), (18.4573672, -33.9585957), (18.4573652, -33.958564), (18.4573568, -33.9585404), (18.4573451, -33.9585162), (18.457334, -33.9585017), (18.4573196, -33.9584906), (18.4573008, -33.9584812), (18.457278, -33.9584717), (18.4571747, -33.9584372)]']

prod_select = df_producer_geo.copy()
prod_select = prod_select.loc[~prod_select["geom_id_order"].isin(bad_arcs)]
prod_select = prod_select.loc[prod_select["category"].isin(descriptions)]
display("Number of producers: {}".format(prod_select.shape[0]))

'Number of producers: 13024'

## Generate service network

In [115]:
logging.info("Preparing road network.......\n")
nodes["u"] = nodes["osmid"]
network_info = NetworkExtract(
    test_network,
    test_network_directed,
    nodes,
    networkh5_file,
    round_cost=True,
    key_pois=key_pois,
)

2022-04-12 13:32:13,500 - root - INFO - Preparing road network.......



/Users/ejwillemse/opt/anaconda3/envs/python_RD/lib/python3.8/site-packages/tables/array.py:241: DeprecationWarning:

`np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations



In [116]:
depot = depot_arc_index
ifs = if_arc_index
req_arcs = network_info.arc_consolidation_standard(prod_select)
network_info.load_required_arcs(req_arcs, merge_network=True)
network_info.set_depot_arc(depot)
network_info.set_if_arcs(ifs)
network_info.loc_in_required_arcs()

network_info.extend_required_inverse_arcs()

network_info.check_main_list()
network_info.load_distance_matrix()
network_info.offload_calculations3D()

network_info.network_gdf(prod_select)

logging.info("Done!")

2022-04-12 13:32:14,151 - root - INFO - Load required arcs
2022-04-12 13:32:14,152 - root - INFO - Merging with the network
2022-04-12 13:32:14,188 - root - INFO - Set depot
2022-04-12 13:32:14,205 - root - INFO - Set offload facilities
2022-04-12 13:32:14,217 - root - INFO - Extend required arcs with inverse edge arcs
2022-04-12 13:32:14,309 - root - INFO - Checking master list
2022-04-12 13:32:14,316 - root - INFO - Load distance matrix: 6562 x 6562


/Users/ejwillemse/opt/anaconda3/envs/python_RD/lib/python3.8/site-packages/tables/array.py:241: DeprecationWarning:

`np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

/Users/ejwillemse/opt/anaconda3/envs/python_RD/lib/python3.8/site-packages/tables/leaf.py:544: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.



2022-04-12 13:32:19,984 - root - INFO - Creating nearest neighbour lists
2022-04-12 13:32:22,126 - root - INFO - Calculate offloads: 6562 x 6562 x 2
2022-04-12 13:32:26,940 - root - INFO - Done!


## Check network connectivity

In [117]:
cut_off = 100000
bad_arcs = []
for i in range(len(network_info.d)):
    if network_info.d[i, 0] > cut_off:
        bad_arc = network_info.reqArcList[i]
        bad_links = test_network.loc[test_network["arc_index"] == bad_arc]
        bad_arcs += bad_links["geom_id_order"].tolist()
len(bad_arcs)
bad_arcs

0

[]

In [118]:
cut_off = 100000
bad_arcs = []
for i in range(len(network_info.d)):
    if network_info.d[0, i] > cut_off:
        bad_arc = network_info.reqArcList[i]
        bad_links = test_network.loc[test_network["arc_index"] == bad_arc]
        bad_arcs += bad_links["geom_id_order"].tolist()
len(bad_arcs)
bad_arcs

0

[]

In [119]:
d_max = network_info.d.max()
if d_max == np.inf:
    d_i = network_info.d.argmax(axis=0)[0]
    bad_arc = network_info.reqArcList[d_i]
    print(network_info.d[d_i, :])
    print(network_info.d[:, d_i])
    d_i2 = network_info.d[d_i, :].argmax()
    bad_arc2 = network_info.reqArcList[d_i2]
    print(network_info.d[d_i2, :])
    print(network_info.d[:, d_i2])
    test_network.loc[test_network["arc_index"] == bad_arc]
    test_network.loc[test_network["arc_index"] == bad_arc2]

## Set scenario input parameters

In [121]:
scenario_name = "Landed refuse"
schedule_select = "weekly"
offload_time = 15 * 60
max_duration = 10.5 * 3600
capacity = 10.5 * 1000
travel_speed = 45 / 3.6
service_speed = 30 / 3.6

demand_increase_factor = {"daily": 1, "weekly": 1, "two_one": 1}
increase_factor = demand_increase_factor[schedule_select]

logging.info("Uploading scenario inputs...")
prod_select_update = prod_select.copy()
prod_arc_update = prod_select_update.copy()

network_info.producer_demand = prod_arc_update
prod_arc_update = network_info.add_demand_service_cost_to_ars(
    prod_select_update, service_speed
)
_ = pd.DataFrame(
    prod_arc_update.drop(columns=["arc_index"]).sum(numeric_only=True)
).astype(int)

network_info.set_travel_speed(travel_speed)
network_info.set_service_cost_and_demand(prod_arc_update)
network_info.set_offload_time(offload_time)
network_info.check_shapes()

network_info.update_cost_matrix()
network_info.update_offload_cost()

network_info.set_vechile_capacity_constraint(capacity)
network_info.set_vehicle_duration_constraint(max_duration)
logging.info("Done")

2022-04-12 13:53:15,738 - root - INFO - Uploading scenario inputs...
2022-04-12 13:53:15,760 - root - INFO - Setting service cost for network
2022-04-12 13:53:15,776 - root - INFO - Setting demand for network
2022-04-12 13:53:15,792 - root - INFO - Update travel durations.
2022-04-12 13:53:15,971 - root - INFO - Update offload durations.
2022-04-12 13:53:16,213 - root - INFO - Done


## Solve problem

In [123]:
solution_df = solve_store_instance(
    "",
    improve=None,
    write_results=False,
    info=network_info,
    overwrite=True,
    test_solution=True,
    full_output=False,
    tollerance=60,
    nnFracLS=1,
    nnFracTS=0.25,
)
logging.info("Done generating routes")
n_route = solution_df["route"].max() + 1
print(f"Number of routes: {n_route}")

2022-04-12 13:53:22,285 - root - INFO - Problem info supplied. Directly proceeding to solve problem.
Test route 0
Test trip 0
Test trip 1
Test trip 2
Test route 1
Test trip 0
Test trip 1
Test trip 2
Test route 2
Test trip 0
Test trip 1
Test route 3
Test trip 0
Test trip 1
Test route 4
Test trip 0
2022-04-12 13:53:47,354 - root - INFO - Done generating routes
Number of routes: 5


## Display KPIs

In [124]:
logging.info("Preparing scenario outputs...")

network_info.extend_prop_info()
network_info.add_solution(solution_df, order=False)
network_info.deconstruct_solution()

network_info.vehicle_collection_schedule(schedule=schedule_select)
network_info.extract_wide_offload_totals()

network_info.add_traversal_time()
network_info.add_time_formatted()
network_info.add_constant_duration_time()

2022-04-12 13:53:47,406 - root - INFO - Preparing scenario outputs...


/Users/ejwillemse/opt/anaconda3/envs/python_RD/lib/python3.8/site-packages/tables/array.py:241: DeprecationWarning:

`np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations



In [125]:
_ = network_info.full_producer_report()

route_summary = RouteSummary(network_info)
route_sum_table = route_summary.route_summary()
logging.info("Done preparing output")
display(route_sum_table)

2022-04-12 13:53:55,733 - root - INFO - Done preparing output


,,Vehicle,Route,Collection day,Offloads,Bins collected,Units served,Demand collected (ton),Time collecting (h),Time travelling (h),Time at treatment facility (h),Route duration (h),Traveling distance (km),Collecting distance (km),Total route distance (km),Electrical consumption (kWh),Tons disposed at @ Offload 2,Tons disposed at @ Offload 3
0,,1,1,Mon,3,3426,3426,24.02,8.69,1.06,0.75,10.54,49.77,89.25,139.02,274.26,0.00,24.02
1,,1,2,Tue,3,3334,3334,23.24,8.21,1.54,0.75,10.52,70.14,79.69,149.83,278.41,21.00,2.24
2,,1,3,Wed,2,2878,2878,20.19,8.07,1.93,0.50,10.48,86.23,98.16,184.39,330.07,0.00,20.19
3,,1,4,Thu,2,2957,2957,20.75,7.56,2.43,0.50,10.41,105.69,79.07,184.75,316.08,10.50,10.25
4,,1,5,Fri,1,429,429,3.03,1.13,0.79,0.25,2.17,35.12,12.59,47.71,71.86,3.03,0.00
5,Total,1,5,,11,13024,13024,91.23,33.67,7.75,2.75,44.13,346.94,358.76,705.70,1270.70,34.53,56.70


## Show routes

In [143]:
route_plot = network_info.df_solution_full.copy()
route_plot["route"] = route_plot["route"] + 1
smaller_10 = route_plot["route"] < 10
route_plot.loc[smaller_10, "route"] = "Route 0" + route_plot.loc[smaller_10][
    "route"
].astype(int).astype(str)
route_plot.loc[~smaller_10, "route"] = "Route " + route_plot.loc[~smaller_10][
    "route"
].astype(int).astype(str)

route_plot["width"] = 0
route_plot.loc[route_plot["activity_type"] == "collect", "width"] = 1

route_plot["collection"] = 0
route_plot.loc[route_plot["activity_type"] == "collect", "width"] = 1

route_plot = gpd.GeoDataFrame(route_plot, geometry="geometry", crs="EPSG:4326")

In [147]:
route_map = Kepler(
    height=800,
    data={
        "routes": route_plot,
        "depot": key_depot_select,
        "disposal": key_if_select,
        "producers": prod_select,
    },
    config_path="config/route_complete_rel.cf",
)

display(route_map.get_render())

2022-04-12 14:33:46,438 - root - INFO - Start preparation of render parameters...
2022-04-12 14:33:46,439 - root - INFO - Loading map config from file: config/route_complete_rel.cf
2022-04-12 14:33:46,505 - root - INFO - Next columns are converted to string: ['geom_id_order', 'arc_category', 'maxspeed', 'u_index-v_index', 'time', 'route', 'activity_type', 'time_const_dur', 'speed']
2022-04-12 14:33:46,517 - root - INFO - Next columns are converted to string: ['geom_id_order', 'geom_id', 'geometry_arc', 'Postal Code', 'type', 'id', 'category', 'arc_id', 'geometry_point', 'highway', 'geometry_arc_snap_point', 'geom_id_inv', 'description', 'geometry_u']
2022-04-12 14:33:46,523 - root - INFO - Next columns are converted to string: ['geom_id_order', 'geom_id', 'geometry_arc', 'Postal Code', 'type', 'id', 'category', 'arc_id', 'geometry_point', 'highway', 'geometry_arc_snap_point', 'geom_id_inv', 'description', 'geometry_u']
2022-04-12 14:33:46,550 - root - INFO - Next columns are converted 

KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [{'dataId': ['routes'], 'id': 'buyx8p8d',…

In [148]:
route_map.save_config("config/route_complete_rel.cf", overwrite_config=True)

2022-04-12 14:33:56,190 - root - WARNING - File already exists and will be over-written.


In [149]:
route_map.save_render("maps/route_complete_rel.html", overwrite_html=True)

2022-04-12 14:33:57,313 - root - WARNING - Such file exists and will be over-written.
2022-04-12 14:33:57,351 - root - INFO - Next columns are converted to string: ['geom_id_order', 'arc_category', 'maxspeed', 'u_index-v_index', 'time', 'route', 'activity_type', 'time_const_dur', 'speed']
2022-04-12 14:33:57,354 - root - INFO - Next columns are converted to string: ['geom_id_order', 'geom_id', 'geometry_arc', 'Postal Code', 'type', 'id', 'category', 'arc_id', 'geometry_point', 'highway', 'geometry_arc_snap_point', 'geom_id_inv', 'description', 'geometry_u']
2022-04-12 14:33:57,358 - root - INFO - Next columns are converted to string: ['geom_id_order', 'geom_id', 'geometry_arc', 'Postal Code', 'type', 'id', 'category', 'arc_id', 'geometry_point', 'highway', 'geometry_arc_snap_point', 'geom_id_inv', 'description', 'geometry_u']
2022-04-12 14:33:57,377 - root - INFO - Next columns are converted to string: ['pipedWater', 'geom_id_order', 'geom_id', 'geometry_arc_snap_point', 'housingType',

In [ ]:
# df_visual.save_config(route_map, 'kepler_configs/route_complete_rel.cf', overwrite=True)
map_data = {
    "routes": route_plot.copy(),
    "depot": key_depot_select.copy(),
    "disposal": key_if_select.copy(),
    "producers": prod_select.copy(),
}
df_visual.save_html(
    route_map,
    data=map_data,
    path=f"maps/{scenario_name}.html",
    overwrite=True,
    aws_host_save=True,
)

## Save output

In [ ]:
logging.info("Writing outup files...")
r = plot_routes(
    network_info,
    key_pois,
    mapbox_key,
    customers=prod_select,
    cust_size={
        "radiusMinPixels": 0.75,
        "radiusMaxPixels": 10,
        "get_radius": 2,
        "color": "lightgrey",
    },
)  # , prod_select
network_info.add_traversal_time()
network_info.add_time_formatted()
network_info.add_constant_duration_time()
_ = network_info.full_producer_report()
network_info.store_results(scenario_name, prod_select, route_sum_table, r)